In [2]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np 
import os 
import json 
import torch 

sns.set_theme(style="whitegrid")
sns.set_context("paper", font_scale=1.5)


/home/bumjin/anaconda3/lib/python3.11/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [22]:
exp_name = "exp_1_not"
dir_path = f"results/{exp_name}/General_Binary"
models = [
          'openai--gpt-4o-mini', 
          'llama3_1_instruct--70b', 
          'llama3_1--8b', 
          'gemma2--9b', 
        #   'exaone--8b', 
          'qwen2--7b'
          ]
result_dfs = []
for model in models:
    result_df = pd.read_csv(os.path.join(dir_path, f"{model}.csv"))
    result_df['model'] = model
    result_dfs.append(result_df)

def mv_func(x):
    values = ['must', 'should', 'ought']
    for v in values:
        if v in x:
            return v 
    if "has to" in x or "have to" in x:
        return "have to"
    return "none"

def process_model_output(x):
    x =str(x)
    if "1" in x and "0" in x:
        return 2
    elif "1" in x:
        return 1
    elif "0" in x:
        return 0
    else:
        return -1 
result_df = pd.concat(result_dfs)
result_df['mv'] = result_df['input'].apply(mv_func)
result_df['model_output'] =  result_df['model_output_raw'].apply(process_model_output)
result_df.head()

targets = ['input_type', 'model', 'mv', 'model_output']
groupby = targets[:-1]
result_averaged = result_df[targets].groupby(groupby).mean().reset_index()
mean = result_averaged[['input_type', 'mv', 'model_output']].groupby(['input_type', 'mv']).mean().reset_index()
var  = result_averaged[['input_type', 'mv', 'model_output']].groupby(['input_type', 'mv']).var().reset_index()
mv_list = ['must', 'should', 'ought', 'have to', ]
# result_averaged
non_mean = mean[(mean['input_type'] == 'none') & (mean['mv'] == 'none')]['model_output'].item()
non_var = var[(var['input_type'] == 'none') & (var['mv'] == 'none')]['model_output'].item()
new_df = pd.DataFrame(columns=['type'] + mv_list)
for input_type in ['strong',]:
    new_row = [input_type]
    for mv in mv_list:
        try:
            m = mean[(mean['input_type'] == input_type) & (mean['mv'] == mv)]['model_output'].item()
            v = var[(var['input_type'] == input_type) & (var['mv'] == mv)]['model_output'].item()
        except Exception as e:
            print(e)
            m = 0
            v = 0
        str_format = f"{non_mean:.2f} ({non_var:.2f}) -> {m:.2f} ({v:.2f})"
        new_row.append(str_format)
    new_df.loc[len(new_df)] = new_row
new_df
# new_df.to_csv(f'averaged_{exp_name}.csv', index=False)

can only convert an array of size 1 to a Python scalar


,type,must,should,ought,have to
0,strong,0.36 (0.14) -> 0.88 (0.01),0.36 (0.14) -> 0.78 (0.03),0.36 (0.14) -> 0.90 (0.01),0.36 (0.14) -> 0.00 (0.00)
